In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from ase.io import read as ase_read
from sklearn.decomposition import PCA
import random
%matplotlib widget

In [ ]:
dataset_path = '../datasets/40-cspbbr3-all.xyz'

In [ ]:
atoms = ase_read(dataset_path, index=f':')
print(len(atoms))

In [ ]:
atoms_sample = random.sample(atoms, k=1000)
# atoms_sample = atoms[:10000]

In [ ]:
X = np.array([a.get_positions().flatten() for a in atoms_sample])
F =  np.array([a.get_forces().flatten() for a in atoms_sample])
E =  np.array([a.get_potential_energy() for a in atoms_sample])

## PCA!

In [ ]:
pca = PCA(n_components=3)
pca.fit(X)
X_trans = pca.transform(X)

In [ ]:
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
cm = plt.cm.get_cmap('RdYlBu')
c = [(e-np.mean(E))/np.std(E) for e in E]
E_log = np.log(-E)
E_log = E

sc = ax.scatter(X_trans[:,0], X_trans[:, 1],X_trans[:, 2], c=E_log, vmin=min(E_log), vmax=max(E_log), s=1, cmap=cm)
plt.colorbar(sc)
plt.show()

# Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
X = np.array([a.get_positions().flatten() for a in atoms_sample])
F =  np.array([a.get_forces().flatten() for a in atoms_sample])
E =  np.array([a.get_potential_energy() for a in atoms_sample])
pca = PCA(n_components=3)
pca.fit(X)
X_trans = pca.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, E,  test_size=0.25)

In [ ]:
max_depth = 50
tree_regr = DecisionTreeRegressor(max_depth=max_depth)
n_estimators = 300
ada_regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=max_depth),
                          n_estimators=n_estimators, random_state=np.random.RandomState(1))

In [ ]:
tree_regr.fit(X_train, y_train)
# ada_regr.fit(X_train, y_train)

In [ ]:
y_tree = tree_regr.predict(X_test)
# y_ada = ada_regr.predict(X_test)

In [ ]:
# Plot the results
plt.close('all')
fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, sharex=True,sharey=True,
                                    figsize=(12, 6))
x = range(len(E))
rmse = mean_squared_error(y_test, y_tree, squared=False)
ax0.scatter(y_test.flatten(), y_tree.flatten(), c="g", label=f"n_estimators=1, RMSE: {rmse:.5f}", linewidth=2, alpha=0.2)
# rmse = mean_squared_error(y_test, y_ada, squared=False)
# ax1.scatter(y_test, y_ada, c="b", label=f"n_estimators={n_estimators}, RMSE: {rmse:.5f}", linewidth=2, alpha=0.2)

ax0.set_xlabel("data")
ax1.set_xlabel("data")
ax0.set_ylabel("target")
# plt.title("Boosted Decision Tree Regression")
ax0.legend()
ax1.legend()
plt.show()

# Gaussian process

In [ ]:
atoms_sample = random.sample(atoms, k=100)
X = np.array([a.get_positions().flatten() for a in atoms_sample])
F =  np.array([a.get_forces().flatten() for a in atoms_sample])
E =  np.array([a.get_potential_energy() for a in atoms_sample])
pca = PCA(n_components=3)
pca.fit(X)
X_trans = pca.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, F,  test_size=0.25)

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct, ConstantKernel, Matern

In [ ]:
plt.close('all')
kernels = [ConstantKernel() * RBF(length_scale=1.0), ConstantKernel() * (DotProduct(sigma_0=1.0)**2), Matern(nu=2.5)]
fig, axs = plt.subplots(nrows=len(kernels), ncols=1, sharex=True,sharey=True,
                                    figsize=(12, 6))
for i, kernel in enumerate(kernels):
    regr = GaussianProcessRegressor(kernel=kernel)

    # plot the decision function for each datapoint on the grid
    regr.fit(X_train, y_train)
    y_regr = regr.predict(X_test)

    rmse = mean_squared_error(y_test, y_regr, squared=False)
    axs[i].scatter(y_test.flatten(), y_regr.flatten(), c="g", label=f"RMSE: {rmse:.5f}", linewidth=2, alpha=0.2)
    axs[i].legend()
    axs[i].set_title(f'kernel: {kernel}')
    
plt.show()
